In [30]:
import numpy as np
import pandas as pd
import keras
from keras import layers, models
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from google.colab import files

# Esimerkki
'''

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

'''

# Ladataan data CSV-tiedostosta
file_path = "data_from_mysql_where_g16.csv"
data = pd.read_csv(file_path)

# Tarkastetaan datan rakenne
print(data.head())
print(data.columns)

# Suodatetaan pois kaikki rivit, joissa 'sensorvalue_d' on 0
data_filtered = data[data['sensorvalue_d'] != 0]

x_data = data_filtered[['sensorvalue_a', 'sensorvalue_b', 'sensorvalue_c']].values  # x, y, z
y_data = data_filtered['sensorvalue_d'].values # suunta

# Skaalataan syötteet [0, 1] väliin käyttäen StandardScaler
scaler = StandardScaler()
x_data = scaler.fit_transform(x_data.astype('float32'))

# Suuntaa on 6 luokkaa
num_classes = 6
y_data = keras.utils.to_categorical(y_data - 1, num_classes)

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")

# Määritellään malli
model = keras.Sequential(
    [
        keras.Input(shape=(x_train.shape[1],)),  # Syötemuoto (x, y, z)
        layers.Dense(128, activation="relu"),  # Ensimmäinen tiheä kerros
        layers.Dropout(0.3),  # Dropout, jotta ylikoulutus ei tapahdu
        layers.Dense(64, activation="relu"),  # Toinen tiheä kerros
        layers.Dense(32, activation="relu"),  # Kolmas tiheä kerros
        layers.Dense(num_classes, activation="softmax"),  # Lopullinen luokittelukerros
    ]
)

model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Koulutetaan mallia
batch_size = 128
epochs = 100

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping])

# Arvioidaan malli testidatalla
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

   id            timestamp  groupid  from_mac  to_mac  sensorvalue_a  \
0  39  2024-11-14 12:30:54       16         0       0         1510.0   
1  40  2024-11-14 12:30:54       16         0       0         1503.0   
2  41  2024-11-14 12:30:54       16         0       0         1511.0   
3  42  2024-11-14 12:30:55       16         0       0         1509.0   
4  43  2024-11-14 12:30:56       16         0       0         1505.0   

   sensorvalue_b  sensorvalue_c  sensorvalue_d  sensorvalue_e  sensorvalue_f  
0         1405.0         1812.0            0.0            0.0              0  
1         1413.0         1818.0            0.0            0.0              0  
2         1416.0         1814.0            0.0            0.0              0  
3         1412.0         1814.0            0.0            0.0              0  
4         1410.0         1804.0            0.0            0.0              0  
x_train shape: (2210, 3)
x_test shape: (553, 3)


Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_67 (Dense)                     │ (None, 128)                 │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_19 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_68 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_69 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_70 (Dense)                     │ (None, 6)                   │             198 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,046 (43.15 KB)

 Trainable params: 11,046 (43.15 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 106ms/step - accuracy: 0.2842 - loss: 1.7533 - val_accuracy: 0.6290 - val_loss: 1.5082
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6071 - loss: 1.4315 - val_accuracy: 0.6290 - val_loss: 1.1432
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6296 - loss: 1.0862 - val_accuracy: 0.6290 - val_loss: 0.8714
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6462 - loss: 0.8918 - val_accuracy: 0.6018 - val_loss: 0.8025
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6475 - loss: 0.8280 - val_accuracy: 0.6923 - val_loss: 0.7901
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6189 - loss: 0.8395 - val_accuracy: 0.6833 - val_loss: 0.7885
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6482 - loss: 0.8107 - val_accuracy: 0.6923 - val_loss: 0.7804
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6497 - loss: 0.8219 - val_accuracy: 0.6923 -